In [187]:
import os.path 

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pickle 

import base64 
import email 
from bs4 import BeautifulSoup
import csv

import gspread
from google.oauth2.service_account import Credentials

In [188]:
SCOPES = ["https://www.googleapis.com/auth/gmail.send",'https://www.googleapis.com/auth/gmail.readonly','https://mail.google.com/', "https://www.googleapis.com/auth/gmail.metadata", "https://www.googleapis.com/auth/gmail.modify"]
client_secret_file = './credenciales/client_secret_python_gmail.json'

## conexion

In [189]:
creds = None
  
# The file token.pickle contains the user access token. 
# Check if it exists 
if os.path.exists('token.pickle'): 
  
    # Read the token from the file and store it in the variable creds 
    with open('token.pickle', 'rb') as token: 
        creds = pickle.load(token) 
  
# If credentials are not available or are invalid, ask the user to log in. 
if not creds or not creds.valid: 
    if creds and creds.expired and creds.refresh_token: 
        creds.refresh(Request()) 
    else: 
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
        creds = flow.run_local_server(port=0) 
  
    # Save the access token in token.pickle file for the next run 
    with open('token.pickle', 'wb') as token: 
        pickle.dump(creds, token) 
  
# Connect to the Gmail API 
service = build('gmail', 'v1', credentials=creds) 

## requests

#### Ver las etiquetas (LABELS)

In [190]:
results_labels = service.users().labels().list(userId="me").execute().get("labels")
results_labels[0]

{'id': 'CHAT',
 'name': 'CHAT',
 'messageListVisibility': 'hide',
 'labelListVisibility': 'labelHide',
 'type': 'system'}

#### TODO EL LISTADO

In [191]:
# request a list of all the messages 
emailIdList = service.users().messages().list(userId='me').execute().get('messages')
emailIdList[0]

{'id': '18c1c2d8fea5206f', 'threadId': '18c1c2d8fea5206f'}

In [192]:
for email_ids in emailIdList: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

In [193]:
email_parts["labelIds"]

['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX']

#### POR TITULO

In [194]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [195]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

### obtener campos del email

##### Queries para poner dentro del method list (como en el siguiente ejemplo de subject):
  - subject:
  - from:
  - to:
  - cc:
  - bcc:
  - label:
  - has:attachment
  - has:drive
  - has:document
  - has:spreadsheet
  - has:presentation
  - filename:
  - newer_than:2d
  - deliveredto:
  - OR or { }: --> from:amy OR from:david

https://support.google.com/mail/answer/7190?hl=en    --> All the terms and REGEXtation

In [196]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [197]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()
    email_field_from = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'From']
    email_field_to = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'To']
    email_field_subject = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Subject']
    email_field_date = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Date'] 

In [198]:
email_field_date

['Wed, 29 Nov 2023 12:47:52 +0100']

In [199]:
email_field_from

['Alv Sz <alvarosaezsanchez@gmail.com>']

### adjuntos

In [200]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail excel').execute().get('messages')

In [201]:
for email_ids in emailIdList_filter_by_title: 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

In [203]:
for part in email_parts['payload']['parts']:
    if part['filename'] == "GEO_IP_1_comb (1).xlsx":
        
        # Decodificar el contenido del archivo adjunto
        print(part['body'])
        file_data = base64.urlsafe_b64decode(part['body']['data'])

        # Guardar el archivo en el sistema local con la extensión correcta (por ejemplo, '.xlsx' para Excel)
        file_name = part['filename']
        if not file_name.endswith('.xlsx'):
            file_name += '.xlsx'

        with open(file_name, 'wb') as f:
            f.write(file_data)

print(f"El archivo {file_name} se ha descargado exitosamente.")

{'attachmentId': 'ANGjdJ_ews5C5VJ56vaLgMjdsBdgS434beEOQFUPMrhE4Ybjzd2ODd47orTKkYWhSEVMJ5d1M4p6GxmiJ2mUCv1WPkhKVVz1jcWlYABO5P2XF_9_U1DlU5sf-fUJncFM6Do4lE2blbcepZIF50_nMExkSnF6-OWs7lQ6scHsiU2RoFpKXgSe08EGnWu3qS4baUeetNwZyWqrN74cLxQpRG3wo2g3zyJQevXuSCQMp3FQPQs5wrLlpajEShcq_rQx5wOnNXywn0rCjOGSfkCCn5OUf1jyC5U-o9KlvmMyte3hlX0ngFsDn3vnmPC7dxE', 'size': 16620}


KeyError: 'data'

#### escrapear todo el texto del email